<a href="https://colab.research.google.com/github/tokaalaa/Facial-Expression-Recognition/blob/main/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the dependencies
import tensorflow as tf
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split

import gc
gc.enable()
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.python.lib.io import file_io
from skimage.transform import resize
from keras import backend as K
from keras.utils import to_categorical

import os
import time
from PIL import Image
import tensorflow_hub as hub
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)


In [ ]:
# class to load the dataset

def preprocess_input(x):
    x /= 128.
    x -= 1.
    return x

# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
    # dataset: Data path
def get_data(dataset):
    file_stream = file_io.FileIO(dataset, mode='r')
    data = pd.read_csv(file_stream)
    pixels = data['pixels'].tolist()
    images = np.empty((len(data), 75, 75, 3))
    i = 0

    for pixel_sequence in pixels:
        single_image = [float(pixel) for pixel in pixel_sequence.split(' ')]  # Extraction of each single
        single_image = np.asarray(single_image).reshape(48, 48) # Dimension: 48x48
        #single_image = resize(single_image, (12, 12), order = 3, mode = 'constant') # Dimension: 139x139x3 (Bicubic)
        ret = np.empty((48, 48, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image

        if ret.shape[-1] == 4:
           ret = ret[...,:-1]
        hr_size = tf.convert_to_tensor(ret.shape[:-1])
        hr_image = tf.image.crop_to_bounding_box(ret, 0, 0, hr_size[0], hr_size[1])
        hr_image = tf.cast(hr_image, tf.float32)
        ret = tf.expand_dims(hr_image, 0)
        #image = np.asarray(tf.squeeze(ret))
        #image = tf.clip_by_value(image, 0, 255)
        #image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
        #plt.imshow(image)
        #plt.title("before " + str(i))

        fake_image = model(ret)
        #print("finish model " + str(i))
        fake_image = tf.squeeze(fake_image)
        fake_image = np.asarray(fake_image)
        fake_image = tf.image.resize(fake_image, [75,75])
        fake_image = tf.clip_by_value(fake_image, 0, 255)
        fake_image = Image.fromarray(tf.cast(fake_image, tf.uint8).numpy())
        #plt.imshow(fake_image)
        #plt.title("after " + str(i))
        images[i, :, :, :] = fake_image
        i += 1
        #if i == 2:
         #  break
    
    images = preprocess_input(images)
    labels = to_categorical(data['emotion'])

    return images, labels 

class DataLoader():
  #root_dir : the path of the root dirctory of the dataset.
  #task : the task of the classifier (acl, meniscus or abnormal).
  #plane : the type of the image (sagittal, coronal or axial).
  # train_ratio : the ratio to split the dataset to train and validation.
     
  def __init__(self, train_ratio = 0.9, oversampling= False, shuffle = True):
    self.train_ratio = train_ratio
    filname = "/content/drive/My Drive/Facial/fer2013.csv"

    train_data_x, train_data_y  = get_data(filname)
    train_data_x, X_test, train_data_y, y_test = train_test_split(train_data_x, train_data_y, test_size=0.1, random_state=0)
    train_data_x, X_val, train_data_y, y_val = train_test_split(train_data_x, train_data_y, test_size=0.1, random_state=0)    

    train_datagen = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

    # Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
    self.train_generator = train_datagen.flow(
    train_data_x,
    train_data_y,
    batch_size  = 128)

    train_datagen2 = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

    # Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
    self.test_generator = train_datagen2.flow(
    X_test,
    y_test,
    batch_size  = 128)


    train_datagen3 = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

    # Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
    self.valid_generator = train_datagen3.flow(
    X_val,
    y_val,
    batch_size  = 128)


  #return : train generator
  def get_train_generator(self):
    return self.train_generator
 
  #return : validation generator
  def get_valid_generator(self):
    return self.valid_generator
 
  #return : test generator
  def get_test_generator(self):
    return self.test_generator
 
  def __del__(self):
    del self.train_generator
    del self.valid_generator
    del self.test_generator

In [ ]:
#dataLoader = DataLoader()

In [ ]:
#dataLoader = DataLoader()

In [ ]:
# dataLoader = DataLoader()

In [ ]:
# train_generator = dataLoader.get_train_generator()
# image, label = train_generator.__getitem__(0)
# plt.imshow(image[25])


In [ ]:
#test_generator = dataLoader.get_test_generator()
#image2, label2 = test_generator.__getitem__(0)
#plt.imshow(image2[0])


In [ ]:
#valid_generator = dataLoader.get_valid_generator()
#image3, label3 = valid_generator.__getitem__(0)
#plt.imshow(image3[0])
